In [1]:
import re
import sqlite3
from urllib.request import urlopen
from html import unescape
import pandas as pd
import os

In [2]:
def fetch(url):
    """
    매개변수로 전달받을 url을 기반으로 웹 페이지를 추출
    웹 페이지의 Content-Type 헤더를 통해 인코딩 형식 확인
    반환값: str 자료형의 HTML
    """
    f = urlopen(url)
    # HTTP 헤더를 기반으로 인코딩 형식 추출
    encoding = f.info().get_content_charset(failobj="utf-8")
    # 추출한 인코딩 형식을 기반으로 문자열 디코딩
    html = f.read().decode(encoding)
    return html

In [3]:
def scrape(html):
    """
    매개변수 html로 받은 HTML을 기반으로 정규 표현식을 사용해 도서 정보를 추출합니다.
    반환값: 도서(dict) 리스트
    """
    books = []
    # re.findall()을 사용해 도서 하나에 해당하는 HTML을 추출
    for partial_html in re.findall(r'<td class="left"><a.*?</td>', html, re.DOTALL):
        # 도서의 URL을 추출
        url = re.search(r'<a href="(.*?)">', partial_html).group(1)
        url = 'http://www.hanbit.co.kr' + url
        # 태그를 제거해서 도서의 제목 추출
        title = re.sub(r'<.*?>', '', partial_html)
        title = unescape(title)
        books.append(pd.DataFrame({'url': [url], 'title': [title]}))
    return pd.concat(books)

In [4]:
def save(db_path, books):
    with sqlite3.connect(os.path.join('.', db_path)) as con: # sqlite DB 파일이 존재하지 않는 경우 파일생성
        try:
            books.to_sql(name = 'BOOKS_INFO', con = con, index = False, if_exists='append') 
            #if_exists : {'fail', 'replace', 'append'} default : fail
        except Exception as e:
            print(str(e))
    
    query = 'SELECT * FROM BOOKS_INFO'
    df = pd.read_sql(query, con = con)
    return df

In [5]:
html = fetch('http://www.hanbit.co.kr/store/books/full_book_list.html')

In [6]:
df = scrape(html)
df.reset_index(drop=True, inplace=True)
df2 = save('books.db', df)
df2

,url,title
0,http://www.hanbit.co.kr/store/books/look.php?p...,청소년 인문학 수업 - 1권
1,http://www.hanbit.co.kr/store/books/look.php?p...,청소년 인문학 수업 - 2권
2,http://www.hanbit.co.kr/store/books/look.php?p...,처음 시작하는 파이썬(2판)
3,http://www.hanbit.co.kr/store/books/look.php?p...,퀀트 전략을 위한 인공지능 트레이딩
4,http://www.hanbit.co.kr/store/books/look.php?p...,처음 시작하는 딥러닝
...,...,...
95,http://www.hanbit.co.kr/store/books/look.php?p...,"CIA 요원, 최강 비즈니스맨이 되다"
96,http://www.hanbit.co.kr/store/books/look.php?p...,필수 공식으로 하루에 하나씩 정복하는 엑셀 업무 공략집
97,http://www.hanbit.co.kr/store/books/look.php?p...,"IT CookBook, CentOS 리눅스"
98,http://www.hanbit.co.kr/store/books/look.php?p...,2020 부동산 세금 사용설명서
